In [ ]:

# 📌 Instalación de dependencias si es necesario
# !pip install xgboost


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [ ]:

from google.colab import files
uploaded = files.upload()

df = pd.read_csv(next(iter(uploaded)))
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.sort_values(by=['motor_id', 'fecha']).reset_index(drop=True)
df.head()


In [ ]:

variables = ['temperatura', 'vibracion', 'presion', 'rpm', 'horas_operacion', 'consumo_energia']
for var in variables:
    df[f'{var}_mean_7d'] = df.groupby('motor_id')[var].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
    df[f'{var}_std_7d'] = df.groupby('motor_id')[var].transform(lambda x: x.rolling(window=7, min_periods=1).std())


In [ ]:

df = df.dropna()
features = [col for col in df.columns if '_mean_7d' in col or '_std_7d' in col]
target = 'falla_en_15_dias'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:

modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, y_train)

y_pred = modelo_rf.predict(X_test)
y_proba = modelo_rf.predict_proba(X_test)[:, 1]

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
